# Chapter-4 微调大模型

本节学习大纲

- 理解大模型微调方法的原理与作用
- 掌握使用 transformers + peft 框架进行大模型微调的基本流程

在上一节课，我们深入学习了如何通过组织提示词激发 LLM 能力以解决复杂任务。通过组织提示词使 LLM 完成上下文学习来解决某些特定任务是使用 LLM 最高效、便捷的方式，但在某些特定场景下，存在仅通过提示词无法较好完成的任务，例如：

- 高度自定义的复杂任务，较难通过有限的上下文让 LLM 充分理解任务要求；
- 具有较高门槛的垂直领域任务，通用 LLM 不具备充分的领域知识，难以实现专业程度解决；
- 对响应时间、算力要求较高的任务，无法使用大体量 LLM，需要用较小的 LLM 完成高难度任务。

在这样的场景下，我们需要基于目标任务对通用 LLM 进行微调，从而使 LLM 充分学习目标任务，在特定的下游任务上达到较好的表现。

## 4.1 环境配置

本章内容将使用 datasets、transformers、peft 等框架完成从微调数据构造到高效微调模型的整体微调流程。首先，我们进行本章的环境配置。

**注：由于在第二章已讲解本地如何部署并调用大模型，本章内容在第二章搭建环境基础上进行，请同学们首先完成第二章本地部署大模型的环境配置。**

本章需要安装的第三方库如下：

In [1]:
!pip install -q datasets pandas peft

同样，本章使用第二章已下载的 Qwen-3-4B 模型作为微调的基座模型。

## 4.2 微调数据集构造

### 4.2.1 什么是有监督微调（SFT）

目前我们所使用的 LLM 一般经过了预训练、有监督微调、人类反馈强化学习三个步骤的训练。预训练是 LLM 强大能力的根本来源，事实上，LLM 所覆盖的海量知识基本都是源于预训练语料。但是，预训练赋予了 LLM 能力，却还需要第二步将其激发出来。经过预训练的 LLM 好像一个博览群书但又不求甚解的书生，对什么样的偏怪问题，都可以流畅地接出下文，但他偏偏又不知道问题本身的含义，只会“死板背书”。因此，我们还需要第二步来教这个博览群书的学生如何去使用它的知识，也就是 SFT（Supervised Fine-Tuning，有监督微调）。所谓有监督微调，其实就是将输入和输出同时给模型，让他根据输出不断去拟合从输入到输出的逻辑，类似于将问题和答案同时给模型，让模型基于答案学习解决问题的过程。在传统 NLP 时代，我们一般会针对每一个任务对模型进行微调。例如，一个经典的 NLP 任务可能是情感分类，也就是判断一句输入文本是积极情绪还是消极情绪，那么就会构造很多输入文本和其情感判断的数据，让模型去学会如何判断输入文本的情感。

而面对能力强大的 LLM，我们往往不再是在指定下游任务上进行微调，而是选择训练模型的“通用指令遵循能力”，也就是一般通过`指令微调`的方式来进行 SFT。所谓指令微调，即我们训练的输入是各种类型的用户指令，而需要模型拟合的输出则是我们希望模型在收到该指令后做出的回复。例如，我们的一条训练样本可以是：

    input:告诉我今天的天气预报？
    output:根据天气预报，今天天气是晴转多云，最高温度26摄氏度，最低温度9摄氏度，昼夜温差大，请注意保暖哦

也就是说，SFT 的主要目标是让模型从多种类型、多种风格的指令中获得泛化的指令遵循能力，也就是能够理解并回复用户的指令。因此，我们使用指令数据对模型进行 SFT 时，需要告知模型用户的指令和模型遵循这个指令所应该的输出。一般 SFT 所使用的指令数据集包括以下三个键：

```json
{
    "instruction":"即输入的用户指令",
    "input":"执行该指令可能需要的补充输入，没有则置空",
    "output":"即模型应该给出的回复"
}
```

例如，如果我们的指令是将目标文本“今天天气真好”翻译成英文，那么该条样本可以构建成如下形式：

```json
{
    "instruction":"将下列文本翻译成英文：",
    "input":"今天天气真好",
    "output":"Today is a nice day！"
}
```

同时，为使模型能够学习到和预训练不同的范式，在 SFT 的过程中，往往会针对性设置特定格式。例如，LLaMA 的 SFT 格式为：

    ### Instruction:\n{{content}}\n\n### Response:\n

其中的 content 即为具体的用户指令，也就是说，对于每一个用户指令，将会嵌入到上文的 content 部分，这里的用户指令不仅指上例中的 “instruction”，而是指令和输入的拼接，即模型可以执行的一条完整指令。例如，针对上例，LLaMA 获得的输入应该是：

    ### Instruction:\n将下列文本翻译成英文：今天天气真好\n\n### Response:\n

其需要拟合的输出则是：

    ### Instruction:\n将下列文本翻译成英文：今天天气真好\n\n### Response:\nToday is a nice day！


## 4.2.2 构造微调数据集

因此，当我们进行 LLM SFT 以提升 LLM 在指定下游任务的表现时，我们需要将训练数据构造成上述格式，并对数据集进行处理来支持模型微调。接下来，我们以角色扮演任务（要求 LLM 扮演甄嬛，以甄嬛的语气、风格与用户对话）为例，演示如何进行微调数据集构造。

我们使用从《甄嬛传》剧本提取出来的对话作为基准来构建训练数据集。提取出来的对话包括上述格式的键：

- instruction：即对话的上文；
- input：此处置为空；
- output：即甄嬛的回复。

微调数据集可以在此处下载：https://github.com/KMnO4-zx/huanhuan-chat/blob/master/dataset/train/lora/huanhuan.json


In [3]:
# 加载第三方库
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer


In [ ]:

# 将JSON文件转换为CSV文件
df = pd.read_json('./huanhuan.json') # 注意修改
ds = Dataset.from_pandas(df)

ds[0]

In [4]:
import pandas as pd
from datasets import Dataset
df2=pd.read_csv('neike.csv',encoding='gbk')
df2=df2.loc[df2['department']=='神经科']
print(df2.value_counts)
ds2=Dataset.from_pandas(df2[0:500])


<bound method DataFrame.value_counts of      department             title  \
13          神经科  想知道癫痫长期用药的危害有什么？   
64          神经科        患有癫痫病会遗传吗？   
65          神经科     经常发作癫痫病能不能减轻？   
66          神经科       癫痫病的问题怎么办呢？   
67          神经科       什么食物容易诱发癫痫?   
...         ...               ...   
7181        神经科      儿童抽动症会比较好治吗？   
7196        神经科     儿童抽动症早期症状有什么？   
7215        神经科         视神经炎容易复发吗   
7224        神经科        早产儿脑炎多久可缓解   
7225        神经科      看小孩子抽动症医院哪家好   

                                                    ask  \
13            查出来癫痫十几年一直在喝药压但是药三分长期喝药一定是不好癫痫长期用药的危害有什么？   
64    我家老伴的癫痫病，不是天生带给的，而是生了一场特别气愤的病遗留下来的后遗症，很忧虑自家的那几...   
65    医生你好。我从小就患癫痫病，现在没断时间都会复发，而且复发的时候就会昏倒。听他们说还会再次出...   
66    想问一下癫痫病的问题，怎么治疗呢？我这个精神病大概有三年了，当时是由于孩子撒手人寰之后遭到刺...   
67    医生，我最近老感觉身体不自觉着抽动，完全不受控制的，有时候严重的话还会口吐白沫，医生说是患了...   
...                                                 ...   
7181  哥哥家的孩子最近总是爱眨眼睛，刚开始家里人以为是孩子眼睛不舒服，也没太在乎，以为多休息就好了...   
7196  我家孩子最近这段时间好象常常不是煮的，超乎突然性抽畜，而且还响起惊叫的声音

接下来我们需要定义一个数据处理函数，该函数能够使用 Qwen-3-4B 的 tokenizer（分词器，即将自然语言文本转化为向量）对提供的训练文本进行处理并送到模型中进行学习。

如上所说，不同 LLM 存在不同的指令格式，在我们进行训练时，需要遵守 LLM 预定义的指令格式，才能使训练取得较好的效果。我们首先查看一下 Qwen-3-4B 的指令格式：

In [5]:
# 加载模型 tokenizer 
tokenizer = AutoTokenizer.from_pretrained('/home/xrxrxlinux/model/Qwen/Qwen3-0.6B', trust_remote=True)

# 打印一下 chat template
messages = [
    {"role": "system", "content": "===system_message_test==="},
    {"role": "user", "content": "===user_message_test==="},
    {"role": "assistant", "content": "===assistant_message_test==="},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)
print(text)

<|im_start|>system
===system_message_test===<|im_end|>
<|im_start|>user
===user_message_test===<|im_end|>
<|im_start|>assistant
<think>

</think>

===assistant_message_test===<|im_end|>
<|im_start|>assistant



接着，我们基于上文打印的指令格式，完成数据集处理函数：

In [6]:
def process_func(example):
    MAX_LENGTH = 1024 # 设置最大序列长度为1024个token
    input_ids, attention_mask, labels = [], [], [] # 初始化返回值
    # 适配chat_template
    instruction = tokenizer(
        f"<s><|im_start|>system\n现在你是经验丰富的{example['department']}医生<|im_end|>\n" 
        f"<|im_start|>user\n{example['ask'] }<|im_end|>\n"  
        f"<|im_start|>assistant\n<think>\n\n</think>\n\n",  
        add_special_tokens=False   
    )
    response = tokenizer(f"{example['answer']}", add_special_tokens=False)
    # 将instructio部分和response部分的input_ids拼接，并在末尾添加eos token作为标记结束的token
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    # 注意力掩码，表示模型需要关注的位置
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    # 对于instruction，使用-100表示这些位置不计算loss（即模型不需要预测这部分）
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 超出最大序列长度截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
# 使用上文定义的函数对数据集进行处理
tokenized_id = ds2.map(process_func, remove_columns=ds2.column_names)
tokenized_id

Map: 100%|██████████| 500/500 [00:00<00:00, 2589.84 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [8]:
# 最终模型的输入
print(tokenizer.decode(tokenized_id[0]['input_ids']))

<s><|im_start|>system
现在你是经验丰富的神经科医生<|im_end|>
<|im_start|>user
查出来癫痫十几年一直在喝药压但是药三分长期喝药一定是不好癫痫长期用药的危害有什么？<|im_end|>
<|im_start|>assistant
<think>

</think>

根据你叙述的情有癫目前没正式救感觉症状越来越严重。在这种情况建议在医生的指导下治疗癫痫症状。根据你叙述的情长期用药可能会致使过敏反也可能会致使粒细胞减少或智力性痤疮。功能障碍、厌食等。癫痫药物的副作用是众所周知的问题。长期使用卡马西平等抗癫痫药必然会致使记忆失去、反映迟缓和肝肾功能危害。<|endoftext|>


In [9]:
# 最终模型的输出
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"]))))

根据你叙述的情有癫目前没正式救感觉症状越来越严重。在这种情况建议在医生的指导下治疗癫痫症状。根据你叙述的情长期用药可能会致使过敏反也可能会致使粒细胞减少或智力性痤疮。功能障碍、厌食等。癫痫药物的副作用是众所周知的问题。长期使用卡马西平等抗癫痫药必然会致使记忆失去、反映迟缓和肝肾功能危害。<|endoftext|>


## 4.3 高效微调-LoRA 

### 4.4.1 什么是高效微调

目前，主流的模型微调方法包括全量微调与高效微调。全量微调即在 SFT 过程中会更新模型全部的参数，因此需要较大的算力，成本较为高昂。针对全量微调的昂贵问题，业界提出了高效微调的方法，即通过向模型中插入新的层，在微调时仅更新新层的少量参数等方法来降低微调的算力需求。

目前，最主流的高效微调方法是 LoRA 微调。LoRA 微调的基本原理是，向模型中插入一些低秩矩阵层作为 LoRA 微调的目标参数，在微调时仅更新这部分参数。由于 LoRA 微调一般只需要模型在预训练的基础上学会一些较简单的任务，因此即使只更新部分插入参数，仍然能达到较好的微调效果。同时，LoRA 插入的低秩矩阵是对原参数的分解，在推理时，可通过矩阵计算直接将 LoRA 参数合并到原模型，从而避免了推理速度的降低。关于 LoRA 微调的更多原理，可以参考[Happy LLM 中关于 LoRA 微调的部分](https://github.com/datawhalechina/happy-llm/blob/main/docs/chapter6/%E7%AC%AC%E5%85%AD%E7%AB%A0%20%E5%A4%A7%E6%A8%A1%E5%9E%8B%E8%AE%AD%E7%BB%83%E6%B5%81%E7%A8%8B%E5%AE%9E%E8%B7%B5.md)。

因此，LoRA 成为目前微调 LLM 的主流方法，尤其是对于资源受限、有监督训练数据受限的情况下，LoRA 微调往往会成为 LLM 微调的首选方法。

### 4.4.2 如何进行 LoRA 微调

我们使用 peft 库来高效、便捷地实现 LoRA 微调。

In [10]:
# 首先配置 LoRA 参数
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, # 任务类型为 CLM，即 SFT 任务的类型
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # 目标模块，即需要进行 LoRA 微调的模块
    inference_mode=False, # 训练模式
    r=8, # Lora 秩，即 LoRA 微调的维度
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'q_proj', 'k_proj', 'up_proj', 'o_proj', 'v_proj', 'gate_proj', 'down_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)

In [11]:
import torch
# 加载基座模型
model = AutoModelForCausalLM.from_pretrained('/home/xrxrxlinux/model/Qwen/Qwen3-0.6B', device_map="auto",torch_dtype=torch.bfloat16)
# 开启模型梯度检查点能降低训练的显存占用
model.enable_input_require_grads()
# 通过下列代码即可向模型中添加 LoRA 模块
model = get_peft_model(model, config)
config

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/home/xrxrxlinux/model/Qwen/Qwen3-0.6B', revision=None, inference_mode=False, r=8, target_modules={'q_proj', 'k_proj', 'up_proj', 'o_proj', 'v_proj', 'gate_proj', 'down_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)

In [12]:
# 查看 lora 微调的模型参数
model.print_trainable_parameters()

trainable params: 5,046,272 || all params: 601,096,192 || trainable%: 0.8395


本文使用 Swanlab 进行训练的监测，可以便捷地查看训练的 loss 情况、GPU 利用率等。通过以下命令安装 swanlab 第三方库：

In [13]:
# 配置 swanlab
import swanlab
from swanlab.integration.transformers import SwanLabCallback

swanlab.login(api_key='PK2G99ohtiCwdwtOfk2KN', save=False)

# 实例化SwanLabCallback
swanlab_callback = SwanLabCallback(
    project="Qwen3-4B-lora-for-medical-nero", 
    experiment_name="Qwen3-4B-experiment-for-medical-nero"
)

/home/xrxrxlinux/miniconda3/envs/llm_vllm/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

swanlab 的 api key 可以通过登录官网注册账号获得：https://swanlab.cn/

In [14]:
from swanlab.integration.transformers import SwanLabCallback

# 配置训练参数
args = TrainingArguments(
    output_dir="/home/xrxrxlinux/model/Qwen/Qwen_new_medical_nero",
    per_device_train_batch_size=4,        # 大幅降低 batch size
    gradient_accumulation_steps=8,      # 相应增加梯度累积，保持有效 batch size
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
    bf16=True,                            # 开启混合精度训练 (如果你的卡支持)
    #optim="paged_adamw_8bit",             # 使用 8-bit 优化器
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
# 然后使用 trainer 训练即可
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback]
)

In [16]:
# 开始训练
trainer.train()

/home/xrxrxlinux/miniconda3/envs/llm_vllm/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

swanlab: Tracking run with swanlab version 0.6.8

swanlab: Run data will be saved locally in 
/home/xrxrxlinux/new_fold/swanlog/run-20250825_172247-y0kptirxb3yz03j5xky7b

swanlab: 👋 Hi sunny_xrxrx,welcome to swanlab!

...

swanlab: 🏠 View project at https://swanlab.cn/@sunny_xrxrx/Qwen3-4B-lora-for-medical-nero

swanlab: 🚀 View run at https://swanlab.cn/@sunny_xrxrx/Qwen3-4B-lora-for-medical-nero/runs/y0kptirxb3yz03j5xky7b

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,4.249700
20,3.786200
30,3.638200
40,3.556800


TrainOutput(global_step=48, training_loss=3.7486926317214966, metrics={'train_runtime': 127.8442, 'train_samples_per_second': 11.733, 'train_steps_per_second': 0.375, 'total_flos': 749981373825024.0, 'train_loss': 3.7486926317214966, 'epoch': 3.0})

LoRA 微调仅保存微调后的 LoRA 参数，因此推理微调模型需要加载 LoRA 参数并合并：

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/home/xrxrxlinux/model/Qwen/Qwen3-0.6B'# 基座模型参数路径
lora_path = '/home/xrxrxlinux/model/Qwen/Qwen_new_medical_nero/checkpoint-48' # 这里改成你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True)

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

In [24]:
prompt = "从小就一直有羊羔疯这种并不是很严很少会复但是快结婚了就很担心会不会是一种严重的所以想了解几同居后才发觉她患癫痫病经常复发:突然意识丧常伴惊叫、面色青紫、尿失禁、舌咬伤、口吐白沫或血沫、瞳孔散大。羊羔疯要怎么治疗？"
inputs = tokenizer.apply_chat_template(
                                    [{"role": "user", "content": prompt}],
                                    add_generation_prompt=True,
                                    tokenize=True,
                                    return_tensors="pt",
                                    return_dict=True,
                                    enable_thinking=False
                                )
inputs = inputs.to("cuda")


gen_kwargs = {"max_length": 512, "do_sample": True, "top_k": 1, 'repetition_penalty':1.1  }
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

根据你描述的情况，建议去正规的医院实施仔细检查和诊断，然后根据医生的建议进行治疗。如果病情严重的话，需要有及时到专业医院实施手术治疗。


# 课后作业

请大家选择一个 NLP 经典任务（例如情感分类、命名实体识别等），收集该任务的经典训练数据对 Qwen3-4B 进行微调，并评估微调后模型在该任务上的效果



# 后续学习建议

**最重要的事——确定目标**

针对学完了本次课程的同学，可以根据个人背景与学习目标对应选择自己的后续学习方向：

- 对非 CS/DS 专业同学：
  - 目标是**使用** LLM 辅助本专业研究，则掌握上述技能已经能够帮助你使用 LLM 赋能专业研究，建议选择专业研究问题，思考如何使用 LLM 赋能并投入实际实践；
  - 目标是**深入探索 AI+X**道路，则建议进一步学习 LLM 底层原理，掌握更系统、全面的 LLM 知识，并向上拓展学习 LLM 更多的应用形式（例如 RAG、Agents 等），通过不断探索找到专业与 LLM 深度融合的路径：
    - 推荐学习：[Happy LLM](https://github.com/datawhalechina/happy-llm)
- 对 CS/DS 专业的初学同学：
  - 目标是深入学习 LLM 且计划**未来从事相关职业**的同学，建议从 ML、NLP 等方向的底层原理开始学习，同步进行基础知识学习及基于本教程展开一些 LLM 实践（做一些 LLM 项目）；
    - 推荐学习：[南瓜书](https://github.com/datawhalechina/pumpkin-book)、[李宏毅深度学习教程](https://github.com/datawhalechina/leedl-tutorial)、[Happy LLM](https://github.com/datawhalechina/happy-llm)、[Self LLM](https://github.com/datawhalechina/self-llm);
  - 目标是快速掌握 LLM 的同学，建议进一步学习 LLM 底层原理，掌握更系统、全面的 LLM 知识，并向上拓展学习 LLM 更多的应用形式（例如 RAG、Agents 等）:
    - 推荐学习：[Happy LLM](https://github.com/datawhalechina/happy-llm)、[Self LLM](https://github.com/datawhalechina/self-llm);